In [4]:
import collections
import os
import string
import sys

import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/charalambos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
data_path = "../data/txt/"

# Choix d'une décennie et du nombre de clusters

In [7]:
DECADE = '1960'
N_CLUSTERS = 5

# Chargement des fichiers de la décennie

In [8]:
files = [f for f in sorted(os.listdir(data_path)) if f"_{DECADE[:-1]}" in f]

In [9]:
# Exemple de fichiers
files[:15]

['Bxl_1960_Tome_II1_Part_1.txt',
 'Bxl_1960_Tome_II1_Part_2.txt',
 'Bxl_1960_Tome_II1_Part_3.txt',
 'Bxl_1960_Tome_II1_Part_4.txt',
 'Bxl_1960_Tome_II1_Part_5.txt',
 'Bxl_1960_Tome_II1_Part_6.txt',
 'Bxl_1960_Tome_II1_Part_7.txt',
 'Bxl_1960_Tome_II1_Part_8.txt',
 'Bxl_1960_Tome_II2_Part_1.txt',
 'Bxl_1960_Tome_II2_Part_2.txt',
 'Bxl_1960_Tome_II2_Part_3.txt',
 'Bxl_1960_Tome_II2_Part_4.txt',
 'Bxl_1960_Tome_II2_Part_5.txt',
 'Bxl_1960_Tome_II2_Part_6.txt',
 'Bxl_1960_Tome_II2_Part_7.txt']

In [10]:
texts = [open(data_path + f).read() for f in files]

In [11]:
# Exemple de textes
texts[0][:400]

'PUBLICATION\n\nPERIODIQUE\n\nVILLE DE BRUXELLES\n\nAnnée 1960\n\nBULLETIN COMMUNAL\nTOME II "\n_u\n\nImpr. H . 6 M . S C H A U M A N S\nSociété Anonyme\n\nParvis Saint-Gilles, 41 Bruxelles 1960\n\n\x0c\x0cN" 14\n\nCOMPTE\n\nR E N D U D E L A S E A N C E D U 1"\' A O U T\n\n1960\n\nVILLE\n\nDE\n\nBRUXELLES\n\nBULLETIN\n(Publication\n\nCOMMUNAL\npériodique)\n\nA n n é e 1960\n\nCONSEIL\n\nCOMMUNAL\n\nS é a n c e du\n\n1\n\ner\n\naoût\n\n1960.\n\nPRESIDENCE\n\n'

# Vectorisation du texte

In [12]:
def process_text(text, stem=True):
    """ Tokenize text and remove punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    return tokens

In [13]:
vectorizer = TfidfVectorizer(tokenizer=process_text,
                                stop_words=stopwords.words('french'),
                                max_df=0.5,
                                min_df=0.1,
                                lowercase=True)

In [14]:
%time tfidf_vectors = vectorizer.fit_transform(texts)

CPU times: user 1min 15s, sys: 992 ms, total: 1min 16s
Wall time: 1min 17s


In [15]:
tfidf_vectors

<420x6668 sparse matrix of type '<class 'numpy.float64'>'
	with 608083 stored elements in Compressed Sparse Row format>

In [17]:
# Exemple de vecteur TFIDF
pd.Series(
    tfidf_vectors[0].toarray()[0],
    index=vectorizer.get_feature_names()
    ).sort_values(ascending=False)

fer        0.285979
curage     0.210133
chemins    0.159929
tension    0.159280
dma        0.153070
             ...   
munal      0.000000
mur        0.000000
murs       0.000000
musin      0.000000
#          0.000000
Length: 6668, dtype: float64

# Comprendre les vecteurs et leurs "distances"

In [18]:
from scipy.spatial.distance import cosine

In [19]:
cosine([1, 2, 3], [1, 2, 3])

0.0

In [20]:
cosine([1, 2, 3], [1, 2, 2])

0.02004211298777725

In [21]:
cosine([1, 2, 3], [2, 2, 2])

0.07417990022744858

In [22]:
tfidf_array = tfidf_vectors.toarray()

In [23]:
tfidf_array[0]

array([0.        , 0.        , 0.02430379, ..., 0.        , 0.01537096,
       0.00911774])

In [24]:
texts[0][:200]

'PUBLICATION\n\nPERIODIQUE\n\nVILLE DE BRUXELLES\n\nAnnée 1960\n\nBULLETIN COMMUNAL\nTOME II "\n_u\n\nImpr. H . 6 M . S C H A U M A N S\nSociété Anonyme\n\nParvis Saint-Gilles, 41 Bruxelles 1960\n\n\x0c\x0cN" 14\n\nCOMPTE\n\nR E'

In [25]:
tfidf_array[0]

array([0.        , 0.        , 0.02430379, ..., 0.        , 0.01537096,
       0.00911774])

In [26]:
cosine(tfidf_array[0], tfidf_array[1])

0.5890213388262817

# Clustering des vecteurs TFIDF

Article intéressant sur le KMeans clustering :
- https://medium.com/dataseries/k-means-clustering-explained-visually-in-5-minutes-b900cc69d175

In [27]:
km_model = KMeans(n_clusters=N_CLUSTERS)

In [28]:
km_model.fit(tfidf_vectors)

KMeans(n_clusters=5)

In [29]:
clustering = collections.defaultdict(list)

for idx, label in enumerate(km_model.labels_):
    clustering[label].append(files[idx])

In [30]:
pprint(dict(clustering))

{0: ['Bxl_1963_Tome_II1_Part_8.txt',
     'Bxl_1963_Tome_II1_Part_9.txt',
     'Bxl_1963_Tome_II2_Part_2.txt',
     'Bxl_1963_Tome_II2_Part_3.txt',
     'Bxl_1963_Tome_II2_Part_4.txt',
     'Bxl_1963_Tome_II2_Part_5.txt',
     'Bxl_1964_Tome_II1_Part_4.txt',
     'Bxl_1964_Tome_II1_Part_8.txt',
     'Bxl_1964_Tome_II1_Part_9.txt',
     'Bxl_1964_Tome_II2_Part_1.txt',
     'Bxl_1964_Tome_II2_Part_2.txt',
     'Bxl_1964_Tome_II2_Part_3.txt',
     'Bxl_1964_Tome_II2_Part_7.txt',
     'Bxl_1964_Tome_II2_Part_9.txt',
     'Bxl_1965_Tome_II1_Part_2.txt',
     'Bxl_1965_Tome_II1_Part_3.txt',
     'Bxl_1965_Tome_II1_Part_5.txt',
     'Bxl_1965_Tome_II2_Part_1.txt',
     'Bxl_1965_Tome_II2_Part_2.txt',
     'Bxl_1965_Tome_II2_Part_3.txt',
     'Bxl_1965_Tome_II2_Part_5.txt',
     'Bxl_1965_Tome_II2_Part_6.txt',
     'Bxl_1965_Tome_II2_Part_7.txt',
     'Bxl_1965_Tome_II2_Part_8.txt',
     'Bxl_1965_Tome_I_Part_10.txt',
     'Bxl_1965_Tome_I_Part_11.txt',
     'Bxl_1965_Tome_I_Part_7.txt',
     

# Analyse des mots-clés de chaque cluster